#Transfer Learning



### Kort sammanfattning av Rethinking the Inception Architecture for Computer Vision




In [2]:
#import tensorflow_datasets as tfds
import tensorflow as tf 
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd 



In [8]:
#from tensorflow.keras.applications import InceptionV3

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

base_model = tf.keras.applications.InceptionV3(input_shape=(120,120,3), include_top=False, weights='imagenet')

# model=keras.applications.InceptionV3(
#     include_top=True,
#     weights="imagenet",
#     input_tensor=None,
#     input_shape=None,
#     pooling=None,
#     classes=1000,
#     classifier_activation="softmax",
# )

# gör så att base model inte är tränbar - kan fixa detta senare

for layer in base_model.layers:
    layer.trainable=False


#base_model.summary()



In [14]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
# Skapa ett tillägg (efterbearbetning) till Inception-v3 modellen med ett flatten (input), mellanlager och output

# Inspirerat från Medium.com

def CNN_model(learning_rate=0.001, drop_rate=0.3):

    print(drop_rate)
    # default learning rate in Adam, Adam is an optimizor for learning rate, 
    # adam = Adam(learning_rate=learning_rate)

    # model = Sequential(name="CNN_model")
    
    # använd Inception-v3 modellen som input i tränbara modellen (dvs dess output blir input)
    model = tf.keras.layers.Flatten()(base_model.output)
    # lägg till ett dense "hidden" layer
    model = tf.keras.layers.Dense(1024, activation='relu')(model)
    # lägg till dropouts
    model= tf.keras.layers.Dropout(drop_rate)(model)
    # lägg till output layer (node)
    model = tf.keras.layers.Dense(1, activation='sigmoid')(model)

    # gör så att input går till Inception-v3 modellen
    model= tf.keras.Model(base_model.input, model)

    

    model.compile(loss="binary_crossentropy", 
                  optimizer=RMSprop(learning_rate=learning_rate), 
                  metrics=["acc"]
    )

    return model

model = CNN_model(.001, .5)


0.5


In [ ]:
def load_data(pathfile):
    f = open(pathfile, 'rb')
    obj1, obj2 = pickle.load(f)
    f.close()
    arr = np.random.choice(np.arange(0, len(obj1)), size=len(obj1), replace=False)
    obj3=[obj1[i] for i in arr]
    obj4=[obj2[i] for i in arr]
    return [obj3, obj4]

X_train, y_train=load_data("experiment_small_data/train/train.dat")
X_test, y_test=load_data("experiment_small_data/test/test.dat")
X_train2, y_train2 =load_data("experiment_small_data/train/train2.dat")
X_val, y_val=load_data("experiment_small_data/val/validate.dat")

def toOneHot(data):
    code=[]
    for x in data:
        if x=='Dog':
            code.append(0)
        else:
            code.append(1)
    return code

# Y_xxx -> y_train, y_test, y_train2, y_val (experimental small)
# y_xxx -> y_train_tiny, y_test_tiny, y_train2_tiny, y_val_tiny

y_train=toOneHot(y_train)
y_test=toOneHot(y_test)
y_train2=toOneHot(y_train2) 
y_val=np.array(toOneHot(y_val))

In [ ]:
size_x=120
size_y=120

def scale_images(image_arr):
    #output_arr = []
    #for image in image_arr:
        # output_arr.append(cv2.resize(image,(size_x, size_y)))
    output_arr = np.array([cv2.resize(image, (size_x, size_y)) for image in image_arr])
    return output_arr


X_train_scaled = scale_images(X_train)
X_train2_scaled = scale_images(X_train2)
X_test_scaled = scale_images(X_test)
X_val_scaled = scale_images(X_val)


X_train_scaled = X_train_scaled/255
X_train2_scaled = X_train2_scaled/255
X_test_scaled = X_test_scaled/255
X_val_scaled = X_val_scaled/255